In [1]:
import sys
# Verificar si se esta trabajando en el entorno virtual
print(sys.executable)


c:\Users\Cruz\Desktop\Pytho for ETL\Gestion Atenciones\Gestion Atenciones\primerEntregable\Scripts\python.exe


In [2]:
# Instalar  ultima version y complemento
# %pip install -U polars
# %pip install xlsx2csv


In [3]:
import polars as pl

# Ruta del archivo de texto
ruta_historicos = './Tickets/Tickets Historico.txt'

# Lista de nombres de columnas a seleccionar
columnas_seleccionadas = ['Numero Ticket', 'Ubicacion', 'Service Desk', 'Estado', 'Fecha Creacion','Fecha Termino','Fecha Cierre']

# Leer y seleccionar
df_historico = pl.read_csv(ruta_historicos,separator=";")
df_historico = df_historico.select(columnas_seleccionadas)

# Convertir las columnas 'str' a date
df_historico = df_historico.cast({"Fecha Creacion":pl.Date, "Fecha Termino":pl.Date, "Fecha Cierre":pl.Date}
)

# Renombrar columnas
df_historico = df_historico.select(
    pl.col("Numero Ticket").alias("TicketID"), *df_historico.drop("Numero Ticket")
)


print(df_historico)


shape: (9_206, 7)
┌──────────────┬──────────────┬──────────────┬─────────┬──────────────┬──────────────┬─────────────┐
│ TicketID     ┆ Ubicacion    ┆ Service Desk ┆ Estado  ┆ Fecha        ┆ Fecha        ┆ Fecha       │
│ ---          ┆ ---          ┆ ---          ┆ ---     ┆ Creacion     ┆ Termino      ┆ Cierre      │
│ str          ┆ str          ┆ str          ┆ str     ┆ ---          ┆ ---          ┆ ---         │
│              ┆              ┆              ┆         ┆ date         ┆ date         ┆ date        │
╞══════════════╪══════════════╪══════════════╪═════════╪══════════════╪══════════════╪═════════════╡
│ WO0000004122 ┆ AREQUIPA -   ┆ Zona Norte   ┆ Cerrado ┆ 2022-06-01   ┆ 2022-06-14   ┆ 2022-06-14  │
│ 687          ┆ 215000       ┆              ┆         ┆              ┆              ┆             │
│ WO0000004122 ┆ METRO AV.    ┆ Zona Centro  ┆ Cerrado ┆ 2022-06-01   ┆ 2022-06-17   ┆ 2022-06-17  │
│ 649          ┆ WIESSE -     ┆              ┆         ┆              ┆  

In [4]:
ruta_actual = './Tickets/Tickets Actual.csv'

# Leer y seleccionar
df_actual = pl.read_csv(ruta_actual,separator="|")
df_actual = df_actual.select(columnas_seleccionadas)

print(df_actual)

shape: (13_050, 7)
┌──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────────┬─────────────┐
│ Numero       ┆ Ubicacion    ┆ Service Desk ┆ Estado   ┆ Fecha        ┆ Fecha       ┆ Fecha       │
│ Ticket       ┆ ---          ┆ ---          ┆ ---      ┆ Creacion     ┆ Termino     ┆ Cierre      │
│ ---          ┆ str          ┆ str          ┆ str      ┆ ---          ┆ ---         ┆ ---         │
│ str          ┆              ┆              ┆          ┆ str          ┆ str         ┆ str         │
╞══════════════╪══════════════╪══════════════╪══════════╪══════════════╪═════════════╪═════════════╡
│ WO0000004853 ┆ TUPAC AMARU  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 10/01/2023  ┆ 13/01/2023  │
│ 311          ┆ - 191038     ┆              ┆          ┆              ┆             ┆             │
│ WO0000004852 ┆ SAN JUAN DE  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2/01/2023   ┆ 5/01/2023   │
│ 942          ┆ LURIGANCHO - ┆              ┆          ┆              ┆

In [5]:
# Convertir las columnas 'str' a date

df_actual = df_actual.with_columns(pl.col("Fecha Cierre").str.split('/').apply(
        lambda lista: lista[2]+"-"+lista[1]+"-"+lista[0] if lista is not None else None
    ),
    pl.col("Fecha Termino").str.split('/').apply(
        lambda lista: lista[2]+"-"+lista[1]+"-"+lista[0] if lista is not None else None
    )
)
df_actual = df_actual.cast({"Fecha Creacion":pl.Date, "Fecha Termino":pl.Date, "Fecha Cierre":pl.Date}, strict=False)

print(df_actual)

shape: (13_050, 7)
┌──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────────┬─────────────┐
│ Numero       ┆ Ubicacion    ┆ Service Desk ┆ Estado   ┆ Fecha        ┆ Fecha       ┆ Fecha       │
│ Ticket       ┆ ---          ┆ ---          ┆ ---      ┆ Creacion     ┆ Termino     ┆ Cierre      │
│ ---          ┆ str          ┆ str          ┆ str      ┆ ---          ┆ ---         ┆ ---         │
│ str          ┆              ┆              ┆          ┆ date         ┆ date        ┆ date        │
╞══════════════╪══════════════╪══════════════╪══════════╪══════════════╪═════════════╪═════════════╡
│ WO0000004853 ┆ TUPAC AMARU  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-10  ┆ 2023-01-13  │
│ 311          ┆ - 191038     ┆              ┆          ┆              ┆             ┆             │
│ WO0000004852 ┆ SAN JUAN DE  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-02  ┆ 2023-01-05  │
│ 942          ┆ LURIGANCHO - ┆              ┆          ┆              ┆

C:\Users\Cruz\AppData\Local\Temp\ipykernel_25452\2305387383.py:3: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  df_actual = df_actual.with_columns(pl.col("Fecha Cierre").str.split('/').apply(
C:\Users\Cruz\AppData\Local\Temp\ipykernel_25452\2305387383.py:6: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("Fecha Termino").str.split('/').apply(


In [6]:
# Renombrar columnas
df_actual = df_actual.select(
    pl.col("Numero Ticket").alias("TicketID"), *df_actual.drop("Numero Ticket")
)

print(df_actual)

shape: (13_050, 7)
┌──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────────┬─────────────┐
│ TicketID     ┆ Ubicacion    ┆ Service Desk ┆ Estado   ┆ Fecha        ┆ Fecha       ┆ Fecha       │
│ ---          ┆ ---          ┆ ---          ┆ ---      ┆ Creacion     ┆ Termino     ┆ Cierre      │
│ str          ┆ str          ┆ str          ┆ str      ┆ ---          ┆ ---         ┆ ---         │
│              ┆              ┆              ┆          ┆ date         ┆ date        ┆ date        │
╞══════════════╪══════════════╪══════════════╪══════════╪══════════════╪═════════════╪═════════════╡
│ WO0000004853 ┆ TUPAC AMARU  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-10  ┆ 2023-01-13  │
│ 311          ┆ - 191038     ┆              ┆          ┆              ┆             ┆             │
│ WO0000004852 ┆ SAN JUAN DE  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-02  ┆ 2023-01-05  │
│ 942          ┆ LURIGANCHO - ┆              ┆          ┆              ┆

In [7]:
# Filtrar 
df_actual= df_actual.filter(pl.col("TicketID").str.slice(0, 2)=="WO")
print(df_actual)

shape: (13_022, 7)
┌──────────────┬──────────────┬──────────────┬──────────┬──────────────┬─────────────┬─────────────┐
│ TicketID     ┆ Ubicacion    ┆ Service Desk ┆ Estado   ┆ Fecha        ┆ Fecha       ┆ Fecha       │
│ ---          ┆ ---          ┆ ---          ┆ ---      ┆ Creacion     ┆ Termino     ┆ Cierre      │
│ str          ┆ str          ┆ str          ┆ str      ┆ ---          ┆ ---         ┆ ---         │
│              ┆              ┆              ┆          ┆ date         ┆ date        ┆ date        │
╞══════════════╪══════════════╪══════════════╪══════════╪══════════════╪═════════════╪═════════════╡
│ WO0000004853 ┆ TUPAC AMARU  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-10  ┆ 2023-01-13  │
│ 311          ┆ - 191038     ┆              ┆          ┆              ┆             ┆             │
│ WO0000004852 ┆ SAN JUAN DE  ┆ Zona Centro  ┆ Cerrado  ┆ 2023-01-02   ┆ 2023-01-02  ┆ 2023-01-05  │
│ 942          ┆ LURIGANCHO - ┆              ┆          ┆              ┆

In [8]:
# Unir y/o concatenar lasdos bases
df_tickets = pl.concat([df_historico, df_actual])
df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre
str,str,str,str,date,date,date
"""WO000000412268…","""AREQUIPA - 215…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-14,2022-06-14
"""WO000000412264…","""METRO AV. WIES…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-17,2022-06-17
"""WO000000412250…","""NICOLAS AYLLON…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-08,2022-06-08
"""WO000000412251…","""JAUREGUI - 405…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-18,2022-06-18
"""WO000000412274…","""CANTO GRANDE -…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-08,2022-06-08
"""WO000000412251…","""CHIMPU OCLLO -…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-03,2022-06-03
"""WO000000412252…","""MINI AGENCIA B…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-07,2022-06-07
"""WO000000412230…","""IMPERIAL - 255…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-10,2022-06-10
"""WO000000412236…","""AREQUIPA - 215…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-13,2022-06-13


In [9]:
# Eliminar duplicados
df_tickets= df_tickets.group_by("*").agg(pl.col("*").unique())
df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre
str,str,str,str,date,date,date
"""WO000000412250…","""NICOLAS AYLLON…","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-08,2022-06-08
"""WO000000412573…","""MALL AVENTURA …","""Zona Norte""","""Cerrado""",2022-06-03,2022-06-06,2022-06-06
"""WO000000412692…","""DEL AIRE - 193…","""Zona Centro""","""Cerrado""",2022-06-03,2022-06-10,2022-06-10
"""WO000000412465…","""PLAZA CIVICA V…","""Zona Centro""","""Cerrado""",2022-06-03,2022-06-12,2022-06-12
"""WO000000412687…","""AGUAS CALIENTE…","""Zona Norte""","""Cerrado""",2022-06-03,2022-06-21,2022-06-21
"""WO000000412579…","""AREQUIPA - 215…","""Zona Norte""","""Cerrado""",2022-06-03,2022-06-14,2022-06-14
"""WO000000412422…","""MONEY AND TOUR…","""Zona Norte""","""Cerrado""",2022-06-03,2022-06-12,2022-06-12
"""WO000000412514…","""MINI AGENCIA B…","""Zona Centro""","""Rechazado""",2022-06-03,2022-06-06,null
"""WO000000412566…","""LAS VIÑAS - 19…","""Zona Centro""","""Rechazado""",2022-06-03,2022-06-12,null


In [10]:
# Quitar duplicados por condicional
df_tickets=df_tickets.sort("Fecha Creacion", descending = False)
df_tickets_sin_duplicados_condiconal= df_tickets.group_by("TicketID").agg(pl.col("Fecha Creacion").last())
df_tickets_sin_duplicados_condiconal

TicketID,Fecha Creacion
str,date
"""WO000000487022…",2023-01-07
"""WO000000552951…",2023-08-24
"""WO000000558272…",2023-09-08
"""WO000000543762…",2023-07-24
"""WO000000503018…",2023-02-27
"""WO000000514626…",2023-04-13
"""WO000000560473…",2023-09-15
"""WO000000438468…",2022-08-29
"""WO000000439658…",2022-09-03


In [11]:
# Conservar las otras columnas
df_tickets = df_tickets.join(df_tickets_sin_duplicados_condiconal, on=["TicketID","Fecha Creacion"], how="inner")
df_tickets


TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre
str,str,str,str,date,date,date
"""WO000000487022…","""SAN MARTIN DE …","""Zona Centro""","""Cerrado""",2023-01-07,2023-02-01,2023-02-04
"""WO000000552951…","""PRO - 191029""","""Zona Centro""","""Cerrado""",2023-08-24,2023-08-27,2023-08-30
"""WO000000558272…","""EL SOL - 19108…","""Zona Centro""","""Cerrado""",2023-09-08,2023-09-15,2023-09-18
"""WO000000543762…","""SANTA ANITA - …","""Zona Sur""","""Cerrado""",2023-07-24,2023-08-03,2023-08-06
"""WO000000503018…","""SAN JUAN DE LU…","""Zona Centro""","""Cerrado""",2023-02-27,2023-03-03,2023-03-06
"""WO000000514626…","""TUPAC AMARU - …","""Zona Centro""","""Rechazado""",2023-04-13,null,null
"""WO000000560473…","""RAUL FERRERO -…","""Zona Sur""","""Cerrado""",2023-09-15,2023-09-16,2023-09-20
"""WO000000438468…","""MALL AVENTURA …","""Zona Norte""","""Cerrado""",2022-08-29,2022-09-19,2022-09-19
"""WO000000439658…","""PLAZA SAN MART…","""Zona Centro""","""Terminado""",2022-09-03,2022-09-03,null


In [12]:
# Separar la columna ubicacion
df_tickets=df_tickets.with_columns((pl.col("Ubicacion").str.split('-').map_elements(
        lambda lista: lista[0]
    )).alias("Agencia"),
    (pl.col("Ubicacion").str.split('-').map_elements(
        lambda lista: lista[1].strip()
    )).alias("AgenciaID")
)
df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID
str,str,str,str,date,date,date,str,str
"""WO000000487022…","""SAN MARTIN DE …","""Zona Centro""","""Cerrado""",2023-01-07,2023-02-01,2023-02-04,"""SAN MARTIN DE …","""191021"""
"""WO000000552951…","""PRO - 191029""","""Zona Centro""","""Cerrado""",2023-08-24,2023-08-27,2023-08-30,"""PRO ""","""191029"""
"""WO000000558272…","""EL SOL - 19108…","""Zona Centro""","""Cerrado""",2023-09-08,2023-09-15,2023-09-18,"""EL SOL ""","""191082"""
"""WO000000543762…","""SANTA ANITA - …","""Zona Sur""","""Cerrado""",2023-07-24,2023-08-03,2023-08-06,"""SANTA ANITA ""","""191052"""
"""WO000000503018…","""SAN JUAN DE LU…","""Zona Centro""","""Cerrado""",2023-02-27,2023-03-03,2023-03-06,"""SAN JUAN DE LU…","""191017"""
"""WO000000514626…","""TUPAC AMARU - …","""Zona Centro""","""Rechazado""",2023-04-13,null,null,"""TUPAC AMARU ""","""191038"""
"""WO000000560473…","""RAUL FERRERO -…","""Zona Sur""","""Cerrado""",2023-09-15,2023-09-16,2023-09-20,"""RAUL FERRERO ""","""193012"""
"""WO000000438468…","""MALL AVENTURA …","""Zona Norte""","""Cerrado""",2022-08-29,2022-09-19,2022-09-19,"""MALL AVENTURA …","""215011"""
"""WO000000439658…","""PLAZA SAN MART…","""Zona Centro""","""Terminado""",2022-09-03,2022-09-03,null,"""PLAZA SAN MART…","""191004"""


In [13]:
# Cambiaratipo entero

df_tickets = df_tickets.cast({"AgenciaID": pl.Int64})
df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID
str,str,str,str,date,date,date,str,i64
"""WO000000487022…","""SAN MARTIN DE …","""Zona Centro""","""Cerrado""",2023-01-07,2023-02-01,2023-02-04,"""SAN MARTIN DE …",191021
"""WO000000552951…","""PRO - 191029""","""Zona Centro""","""Cerrado""",2023-08-24,2023-08-27,2023-08-30,"""PRO """,191029
"""WO000000558272…","""EL SOL - 19108…","""Zona Centro""","""Cerrado""",2023-09-08,2023-09-15,2023-09-18,"""EL SOL """,191082
"""WO000000543762…","""SANTA ANITA - …","""Zona Sur""","""Cerrado""",2023-07-24,2023-08-03,2023-08-06,"""SANTA ANITA """,191052
"""WO000000503018…","""SAN JUAN DE LU…","""Zona Centro""","""Cerrado""",2023-02-27,2023-03-03,2023-03-06,"""SAN JUAN DE LU…",191017
"""WO000000514626…","""TUPAC AMARU - …","""Zona Centro""","""Rechazado""",2023-04-13,null,null,"""TUPAC AMARU """,191038
"""WO000000560473…","""RAUL FERRERO -…","""Zona Sur""","""Cerrado""",2023-09-15,2023-09-16,2023-09-20,"""RAUL FERRERO """,193012
"""WO000000438468…","""MALL AVENTURA …","""Zona Norte""","""Cerrado""",2022-08-29,2022-09-19,2022-09-19,"""MALL AVENTURA …",215011
"""WO000000439658…","""PLAZA SAN MART…","""Zona Centro""","""Terminado""",2022-09-03,2022-09-03,null,"""PLAZA SAN MART…",191004


In [14]:
# Crear la nueva columna by condicion
df_tickets = df_tickets.with_columns(
    pl.when(pl.col("Fecha Termino").is_null()).then(pl.col("Fecha Cierre")).otherwise(pl.col("Fecha Termino")).alias("Fecha Real Fin")
)
df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID,Fecha Real Fin
str,str,str,str,date,date,date,str,i64,date
"""WO000000487022…","""SAN MARTIN DE …","""Zona Centro""","""Cerrado""",2023-01-07,2023-02-01,2023-02-04,"""SAN MARTIN DE …",191021,2023-02-01
"""WO000000552951…","""PRO - 191029""","""Zona Centro""","""Cerrado""",2023-08-24,2023-08-27,2023-08-30,"""PRO """,191029,2023-08-27
"""WO000000558272…","""EL SOL - 19108…","""Zona Centro""","""Cerrado""",2023-09-08,2023-09-15,2023-09-18,"""EL SOL """,191082,2023-09-15
"""WO000000543762…","""SANTA ANITA - …","""Zona Sur""","""Cerrado""",2023-07-24,2023-08-03,2023-08-06,"""SANTA ANITA """,191052,2023-08-03
"""WO000000503018…","""SAN JUAN DE LU…","""Zona Centro""","""Cerrado""",2023-02-27,2023-03-03,2023-03-06,"""SAN JUAN DE LU…",191017,2023-03-03
"""WO000000514626…","""TUPAC AMARU - …","""Zona Centro""","""Rechazado""",2023-04-13,null,null,"""TUPAC AMARU """,191038,null
"""WO000000560473…","""RAUL FERRERO -…","""Zona Sur""","""Cerrado""",2023-09-15,2023-09-16,2023-09-20,"""RAUL FERRERO """,193012,2023-09-16
"""WO000000438468…","""MALL AVENTURA …","""Zona Norte""","""Cerrado""",2022-08-29,2022-09-19,2022-09-19,"""MALL AVENTURA …",215011,2022-09-19
"""WO000000439658…","""PLAZA SAN MART…","""Zona Centro""","""Terminado""",2022-09-03,2022-09-03,null,"""PLAZA SAN MART…",191004,2022-09-03


In [15]:
# Calculamos diferencia de fechas en dias
df_tickets = df_tickets.with_columns(( pl.col("Fecha Real Fin") - pl.col("Fecha Creacion") ).dt.total_days().alias("Dias Cierre"))

df_tickets

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID,Fecha Real Fin,Dias Cierre
str,str,str,str,date,date,date,str,i64,date,i64
"""WO000000487022…","""SAN MARTIN DE …","""Zona Centro""","""Cerrado""",2023-01-07,2023-02-01,2023-02-04,"""SAN MARTIN DE …",191021,2023-02-01,25
"""WO000000552951…","""PRO - 191029""","""Zona Centro""","""Cerrado""",2023-08-24,2023-08-27,2023-08-30,"""PRO """,191029,2023-08-27,3
"""WO000000558272…","""EL SOL - 19108…","""Zona Centro""","""Cerrado""",2023-09-08,2023-09-15,2023-09-18,"""EL SOL """,191082,2023-09-15,7
"""WO000000543762…","""SANTA ANITA - …","""Zona Sur""","""Cerrado""",2023-07-24,2023-08-03,2023-08-06,"""SANTA ANITA """,191052,2023-08-03,10
"""WO000000503018…","""SAN JUAN DE LU…","""Zona Centro""","""Cerrado""",2023-02-27,2023-03-03,2023-03-06,"""SAN JUAN DE LU…",191017,2023-03-03,4
"""WO000000514626…","""TUPAC AMARU - …","""Zona Centro""","""Rechazado""",2023-04-13,null,null,"""TUPAC AMARU """,191038,null,null
"""WO000000560473…","""RAUL FERRERO -…","""Zona Sur""","""Cerrado""",2023-09-15,2023-09-16,2023-09-20,"""RAUL FERRERO """,193012,2023-09-16,1
"""WO000000438468…","""MALL AVENTURA …","""Zona Norte""","""Cerrado""",2022-08-29,2022-09-19,2022-09-19,"""MALL AVENTURA …",215011,2022-09-19,21
"""WO000000439658…","""PLAZA SAN MART…","""Zona Centro""","""Terminado""",2022-09-03,2022-09-03,null,"""PLAZA SAN MART…",191004,2022-09-03,0


In [16]:
condiciones_dias_cierre = [
    
    (pl.col("Dias Cierre").is_null(), None),
    (pl.col("Dias Cierre") <= 3, pl.lit("0 a 3 días")),
    (pl.col("Dias Cierre") <= 7, pl.lit("4 a 7 días")),
    (pl.col("Dias Cierre") <= 15, pl.lit("8 a 15 días")),
    (pl.col("Dias Cierre") > 15, pl.lit("+15 días")),
]

df_tickets = df_tickets.with_columns(
    pl.when(condiciones_dias_cierre[0][0]).then(condiciones_dias_cierre[0][1])
    .when(condiciones_dias_cierre[1][0]).then(condiciones_dias_cierre[1][1])
    .when(condiciones_dias_cierre[2][0]).then(condiciones_dias_cierre[2][1])
    .when(condiciones_dias_cierre[3][0]).then(condiciones_dias_cierre[3][1])
    .when(condiciones_dias_cierre[4][0]).then(condiciones_dias_cierre[4][1])
    # Agrega más condiciones según sea necesario
    .otherwise(None).alias("Grupo Dias")
)
df_tickets.sort("TicketID")

TicketID,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID,Fecha Real Fin,Dias Cierre,Grupo Dias
str,str,str,str,date,date,date,str,i64,date,i64,str
"""OT000000484937…","""NAYLAMP - 1911…","""Zona Centro""","""Cerrado""",2022-12-31,2023-01-25,2023-01-28,"""NAYLAMP """,191143,2023-01-25,25,"""+15 días"""
"""OT000000484938…","""LOS DAMASCOS -…","""Zona Sur""","""Cerrado""",2022-12-31,2023-01-04,2023-01-07,"""LOS DAMASCOS """,193038,2023-01-04,4,"""4 a 7 días"""
"""OT000000484972…","""LUREN - 380004…","""Zona Sur""","""Cerrado""",2022-12-31,2023-01-02,2023-01-05,"""LUREN """,380004,2023-01-02,2,"""0 a 3 días"""
"""OT000000484973…","""EL PINO - 1910…","""Zona Centro""","""Cerrado""",2022-12-31,2023-01-06,2023-01-09,"""EL PINO """,191087,2023-01-06,6,"""4 a 7 días"""
"""SA000000484930…","""TUSILAGOS - 19…","""Zona Centro""","""Rechazado""",2022-12-31,null,null,"""TUSILAGOS """,191108,null,null,null
"""WO000000412230…","""IMPERIAL - 255…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-10,2022-06-10,"""IMPERIAL """,255001,2022-06-10,9,"""8 a 15 días"""
"""WO000000412230…","""IMPERIAL - 255…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-17,2022-06-17,"""IMPERIAL """,255001,2022-06-17,16,"""+15 días"""
"""WO000000412235…","""PLAZA UNION - …","""Zona Centro""","""Cerrado""",2022-06-01,2022-06-05,2022-06-05,"""PLAZA UNION """,191062,2022-06-05,4,"""4 a 7 días"""
"""WO000000412236…","""AREQUIPA - 215…","""Zona Norte""","""Cerrado""",2022-06-01,2022-06-13,2022-06-13,"""AREQUIPA """,215000,2022-06-13,12,"""8 a 15 días"""


In [17]:
#%pip install xlsx2csv

In [18]:
import os
from xlsx2csv import Xlsx2csv
#Importar excels de manera masiva?
dataframes_xlsx = []
directorio="./Atenciones"
#excelss=pl.read_excel(source="C:/Users/ycruz003/Desktop/pythonETL/Gestion Atenciones/Atenciones/Atenciones_Centro.xlsx")
dtypes = {
    "Numero Ticket": pl.Utf8,
    "Tipo de Ticket": pl.Utf8,
    "Proveedor": pl.Utf8,  # Agregar la columna "COSTO CERO" y su tipo de dato
    "Costo Atencion":pl.Utf8
}
null_values = ["", "NA", "N/A", "null", "NaN", "None", "COSTO CERO"]

for archivo in os.listdir(directorio):
    if archivo.endswith('.xlsx'):
        # Ruta completa del archivo Excel
        ruta_excel = os.path.join(directorio, archivo)
        # Nombre del archivo CSV (mismo nombre pero con extensión .csv)
        nombre_csv = os.path.splitext(archivo)[0] + '.csv'
        # Ruta completa del archivo CSV
        ruta_csv = os.path.join(directorio, nombre_csv)
        # Convertir el archivo Excel a CSV con xlsx2csv
        Xlsx2csv(ruta_excel, outputencoding="utf-8").convert(ruta_csv)
        # Leer el archivo CSV con Polars
        df_csv = pl.read_csv(ruta_csv, columns=["Numero Ticket", "Tipo de Ticket", "Proveedor","Costo Atencion"],dtypes=dtypes, ignore_errors=True, null_values=null_values)
        # Agregar el DataFrame a la lista
        dataframes_xlsx.append(df_csv)

# Concatenar todos los DataFrames en uno solo
df_atenciones = pl.concat(dataframes_xlsx)
df_atenciones


Numero Ticket,Tipo de Ticket,Proveedor,Costo Atencion
str,str,str,str
"""WO000000558468…","""Variable""","""MR EXPRESS""",null
"""WO000000454494…","""Rechazado""",null,"""100"""
"""WO000000454524…","""Rechazado""",null,"""100"""
"""WO000000455912…","""Rechazado""",null,"""100"""
"""WO000000470747…","""Rechazado""",null,"""100"""
"""WO000000475997…","""Rechazado""",null,"""100"""
"""WO000000479587…","""Rechazado""",null,"""100"""
"""WO000000482530…","""Rechazado""",null,"""100"""
"""WO000000485368…","""Flat""","""ACCENTURE""","""100"""


In [19]:
# Cambiar el tipo
df_atenciones=df_atenciones.select( pl.col("Numero Ticket").alias("TicketID"), *df_atenciones.drop("Numero Ticket"))
df_atenciones.sort("TicketID")

TicketID,Tipo de Ticket,Proveedor,Costo Atencion
str,str,str,str
"""WO000000412230…","""Flat""","""ACCENTURE""",null
"""WO000000412230…","""Flat""","""ACCENTURE""",null
"""WO000000412235…","""Flat""","""ACCENTURE""",null
"""WO000000412236…","""Flat""","""ACCENTURE""",null
"""WO000000412238…","""Rechazado""",null,null
"""WO000000412240…","""Flat""","""ACCENTURE""",null
"""WO000000412241…","""Rechazado""",null,null
"""WO000000412242…","""Variable""","""PROSEGUR""",null
"""WO000000412244…","""Variable""","""PROSEGUR""",null


In [20]:
# Uppercase a Costo Atencion
df_atenciones=df_atenciones.with_columns(pl.col("Costo Atencion").str.to_uppercase())

df_atenciones=df_atenciones.with_columns(pl.col("Costo Atencion").str.replace(',','.'))

df_atenciones = df_atenciones.with_columns(
    (pl.when(df_atenciones['Costo Atencion'] == 'SIN COSTO').then(pl.lit('0')).otherwise(df_atenciones['Costo Atencion'])).alias("Costo Atencion"),
    
)
df_atenciones = df_atenciones.with_columns(
    (pl.when(df_atenciones['Costo Atencion'] == 'COSTO CERO').then(pl.lit('0')).otherwise(df_atenciones['Costo Atencion'])).alias("Costo Atencion")
)

# Convertir a decimal Costo Atencion
df_atenciones=df_atenciones.cast({"Costo Atencion":pl.Float64}, strict=False)
df_atenciones

TicketID,Tipo de Ticket,Proveedor,Costo Atencion
str,str,str,f64
"""WO000000558468…","""Variable""","""MR EXPRESS""",null
"""WO000000454494…","""Rechazado""",null,100.0
"""WO000000454524…","""Rechazado""",null,100.0
"""WO000000455912…","""Rechazado""",null,100.0
"""WO000000470747…","""Rechazado""",null,100.0
"""WO000000475997…","""Rechazado""",null,100.0
"""WO000000479587…","""Rechazado""",null,100.0
"""WO000000482530…","""Rechazado""",null,100.0
"""WO000000485368…","""Flat""","""ACCENTURE""",100.0


In [21]:
df_atenciones=df_atenciones.group_by("*").agg(pl.col("*").unique())
df_atenciones

TicketID,Tipo de Ticket,Proveedor,Costo Atencion
str,str,str,f64
"""WO000000536311…","""Variable""","""ACCENTURE""",100.0
"""WO000000539853…","""Flat""","""ACCENTURE""",100.0
"""WO000000551838…","""Flat""","""ACCENTURE""",100.0
"""WO000000559616…","""Flat""","""ACCENTURE""",100.0
"""WO000000533136…","""Emergencia""","""PUNTO CONTROL …",1026.6
"""WO000000552300…","""Emergencia""","""PUNTO CONTROL …",1028.37
"""WO000000507411…","""Variable""","""COORPORACION R…",103.84
"""WO000000530592…","""Flat""","""ACCENTURE""",105.0
"""WO000000554546…","""Variable""","""GOICOCHEA MADI…",106.2


In [22]:
#Juntar los dos dataframes
df_final=df_atenciones.join(df_tickets, on="TicketID", how="inner")
df_final

TicketID,Tipo de Ticket,Proveedor,Costo Atencion,Ubicacion,Service Desk,Estado,Fecha Creacion,Fecha Termino,Fecha Cierre,Agencia,AgenciaID,Fecha Real Fin,Dias Cierre,Grupo Dias
str,str,str,f64,str,str,str,date,date,date,str,i64,date,i64,str
"""WO000000536311…","""Variable""","""ACCENTURE""",100.0,"""TARAPACA - 191…","""Zona Centro""","""Cerrado""",2023-07-01,2023-07-07,2023-07-10,"""TARAPACA """,191030,2023-07-07,6,"""4 a 7 días"""
"""WO000000539853…","""Flat""","""ACCENTURE""",100.0,"""PLAZA LIMA NOR…","""Zona Centro""","""Cerrado""",2023-07-13,2023-07-15,2023-07-18,"""PLAZA LIMA NOR…",191058,2023-07-15,2,"""0 a 3 días"""
"""WO000000551838…","""Flat""","""ACCENTURE""",100.0,"""SAN MARTIN DE …","""Zona Centro""","""Cerrado""",2023-08-20,2023-08-21,2023-08-24,"""SAN MARTIN DE …",191021,2023-08-21,1,"""0 a 3 días"""
"""WO000000559616…","""Flat""","""ACCENTURE""",100.0,"""URB SAN FELIPE…","""Zona Centro""","""Cerrado""",2023-09-11,2023-09-17,2023-09-20,"""URB SAN FELIPE…",191053,2023-09-17,6,"""4 a 7 días"""
"""WO000000533136…","""Emergencia""","""PUNTO CONTROL …",1026.6,"""PLAZA LIMA NOR…","""Zona Centro""","""Cerrado""",2023-06-19,2023-06-30,2023-07-03,"""PLAZA LIMA NOR…",191058,2023-06-30,11,"""8 a 15 días"""
"""WO000000552300…","""Emergencia""","""PUNTO CONTROL …",1028.37,"""PLAZA LIMA NOR…","""Zona Centro""","""Cerrado""",2023-08-21,2023-08-27,2023-08-30,"""PLAZA LIMA NOR…",191058,2023-08-27,6,"""4 a 7 días"""
"""WO000000507411…","""Variable""","""COORPORACION R…",103.84,"""MINI AGENCIA B…","""Zona Centro""","""Cerrado""",2023-03-11,2023-04-15,2023-04-18,"""MINI AGENCIA B…",191090,2023-04-15,35,"""+15 días"""
"""WO000000530592…","""Flat""","""ACCENTURE""",105.0,"""NARANJAL - 191…","""Zona Centro""","""Cerrado""",2023-06-11,2023-06-15,2023-06-18,"""NARANJAL """,191141,2023-06-15,4,"""4 a 7 días"""
"""WO000000554546…","""Variable""","""GOICOCHEA MADI…",106.2,"""NAYLAMP - 1911…","""Zona Centro""","""Cerrado""",2023-08-27,2023-09-02,2023-09-05,"""NAYLAMP """,191143,2023-09-02,6,"""4 a 7 días"""


In [23]:
# Definir la estructurade variables del df
df_final=df_final.select("TicketID","AgenciaID","Agencia","Service Desk","Estado",pl.col("Fecha Creacion").dt.strftime("%d/%m/%Y"),pl.col("Fecha Real Fin").dt.strftime("%d/%m/%Y").alias("Fecha Cierre"),"Grupo Dias",pl.col("Tipo de Ticket").alias("Tipo Ticket"), pl.col("Costo Atencion").round(2).alias("Costo"))
df_final

TicketID,AgenciaID,Agencia,Service Desk,Estado,Fecha Creacion,Fecha Cierre,Grupo Dias,Tipo Ticket,Costo
str,i64,str,str,str,str,str,str,str,f64
"""WO000000536311…",191030,"""TARAPACA ""","""Zona Centro""","""Cerrado""","""01/07/2023""","""07/07/2023""","""4 a 7 días""","""Variable""",100.0
"""WO000000539853…",191058,"""PLAZA LIMA NOR…","""Zona Centro""","""Cerrado""","""13/07/2023""","""15/07/2023""","""0 a 3 días""","""Flat""",100.0
"""WO000000551838…",191021,"""SAN MARTIN DE …","""Zona Centro""","""Cerrado""","""20/08/2023""","""21/08/2023""","""0 a 3 días""","""Flat""",100.0
"""WO000000559616…",191053,"""URB SAN FELIPE…","""Zona Centro""","""Cerrado""","""11/09/2023""","""17/09/2023""","""4 a 7 días""","""Flat""",100.0
"""WO000000533136…",191058,"""PLAZA LIMA NOR…","""Zona Centro""","""Cerrado""","""19/06/2023""","""30/06/2023""","""8 a 15 días""","""Emergencia""",1026.6
"""WO000000552300…",191058,"""PLAZA LIMA NOR…","""Zona Centro""","""Cerrado""","""21/08/2023""","""27/08/2023""","""4 a 7 días""","""Emergencia""",1028.37
"""WO000000507411…",191090,"""MINI AGENCIA B…","""Zona Centro""","""Cerrado""","""11/03/2023""","""15/04/2023""","""+15 días""","""Variable""",103.84
"""WO000000530592…",191141,"""NARANJAL ""","""Zona Centro""","""Cerrado""","""11/06/2023""","""15/06/2023""","""4 a 7 días""","""Flat""",105.0
"""WO000000554546…",191143,"""NAYLAMP ""","""Zona Centro""","""Cerrado""","""27/08/2023""","""02/09/2023""","""4 a 7 días""","""Variable""",106.2


In [24]:
# %pip install pandas
# %pip install pyarrow
# %pip install numpy
# %pip install openpyxl

In [25]:
# Se esta utilizando pandas, y todas las lbrerias menionadas arriba puesto que polars ni xlsx2csv tienen la funcion de exportar excels
import pandas as pd
dt_final_pd= df_final.to_pandas()
dt_final_pd.to_excel(
    "./Consolidado.xlsx", sheet_name="Atenciones", index=False
)